# 3. Selected Topics in Numerical Methods

## 3.1 Curve Fitting

Assuming $x_n$ is an input value and $y_i$ is an output value for each $i = 1, ...,n$, discrete data saets are usually represented as the following:
$$ (x_1, y_1), (x_2, y_2),\cdots,(x_n, y_n)$$

We often want to represent $x_n$ and $y_n$ as an analytical function for further analysis such as the linear function:
$$y = \beta_1 + \beta_2x$$

or an exponential function:
$$y = \beta_1e^{\beta_2x}$$

In general, a mathematical function with a vector of parameter $\beta$ can be represented below:
$$y = f(x;\beta)$$

Curve fitting aims to find the relationship between $y_n$ and $x_n$ by finding the values of $\beta$ in $f(x;\beta)$ so that the obtained analytical functional form is the closest to the original discrete data set.

Related to curve fitting, interpolation finds the values of parameters so that the analytical function passes through the discrete data points. This approach assumes that the discrete data set is accurate and exact. When we use polynomial functions for interpolation—called <em>‘polynomial interpolation’</em> —available methods are <b>Lagrange’s Method, Newton’s Method, and Neville’s Method</b>.

However, the most popular definition in curve fitting is based on least-squares. That is, we aim to minimize

$$S(\beta) = \sum_{i=1}^{n} (y_i - f(x_i;\beta))^2$$

by optimally choosing the value $\beta$. This is a nonlinear optimization provelm in general. When we want a linaer function for $f(x)$, commonly called as a linear regression problem, the problem takes the form of a quadratic optimization problem.

Finding an optimal $\beta$ is related to solving a system of equations:
$$\dfrac{\delta S}{\delta\beta_i} = 0 \ \ \ \forall i=1,\cdots,m$$
where $m$ is the number of parameters. For general nonlinear least-square fit, the Levenberg-Marquardt algorithm is popular. See [Nocedal and Wright (2006)](https://link.springer.com/book/10.1007/978-0-387-40065-5) for details. In Julia, the LsqFit Package implements the Levenberg-Marquardt algorithm. The implementation will be shown below:

In [ ]:
# libraries and packages
using LsqFit
using PyPlot

Assuming we have the following dataset:

In [ ]:
# x and y datasets
xdata = [ 15.2; 19.9;  2.2; 11.8; 12.1; 18.1; 11.8; 13.4; 11.5;  0.5; 
          18.0; 10.2; 10.6; 13.8;  4.6;  3.8; 15.1; 15.1; 11.7;  4.2 ];
ydata = [ 0.73; 0.19; 1.54; 2.08; 0.84; 0.42; 1.77; 0.86; 1.95; 0.27;
          0.39; 1.39; 1.25; 0.76; 1.99; 1.53; 0.86; 0.52; 1.54; 1.05 ];


We will be using the curve fitting algorithm on the following function form:
$$f(x) = \beta_1(\dfrac{x}{\beta_2})^{\beta_3-1}exp(-(\dfrac{x}{\beta_2})^{\beta_3})$$

In [78]:
function model(xdata, beta)
    values = similar(xdata)
    for i in 1:length(values)
        values[i] = beta[1]*((xdata[i]/beta[2])^(beta[3]-1))*
                    (exp(-(xdata[i]/beta[2])^beta[3]))
    end
    return values
end

model (generic function with 1 method)

The function above could also be written as below:

In [79]:
# model(x,beta) = beta[1]*((x/beta[2]).^(beta[3]-1)).*
#                     (exp.(-(x/beta[2]).^beta[3]))

Initialising $\beta$ based on $[3.0, 8.0, 3.0]$:

In [80]:
β = [3.0, 8.0, 3.0]
fit = curve_fit(model, xdata, ydata, β)

LsqFit.LsqFitResult{Vector{Float64}, Vector{Float64}, Matrix{Float64}, Vector{Float64}}([4.45941432574688, 10.254403821588156, 1.8911376587725592], [0.04161413385899926, 0.05221091863629376, -0.46866129812660984, -0.7083121787846436, 0.47653739870121836, -0.024386779605210407, -0.39831217878464353, 0.2177192781952021, -0.5237022566232492, 0.03110900165915198, 0.01586650853520727, 0.25919376663315385, 0.33385901078827573, 0.24620338748547232, -0.2374800995099624, 0.04993492123311283, -0.07261618564467287, 0.2673838143553271, -0.15003460112969536, 0.6231233936280016], [0.1730303751761524 0.23249567388207273 -0.3355954261168526; 0.05431451328442292 0.13546244655789239 -0.4020822613435396; … ; 0.31169236525601984 0.20815989727348638 -0.05192522253629633; 0.3751890430856931 -0.08835397600772396 -1.2173622782585123], true, Float64[])

In [81]:
fit.param

3-element Vector{Float64}:
  4.45941432574688
 10.254403821588156
  1.8911376587725592

In [84]:
fig = figure()

xfit = 0:0.1:20
yfit = model(xfit, fit.param)

plot(xdata, ydata, color="black", linewidth=2.0, marker = "o", linestyle = "None")
plot(xfit, yfit, color = "red", linewidth = 2.0)

xlabel("x", fontsize="xx-large")
ylabel("y", fontsize="xx-large")

savefig("fit_plot.png")
close(fig)

## 4.2 Numerical Differentiation

We will be going through Numerical Differentiation for the following equation
$$f(x) = x^3e^x + \sin{x}$$

In [4]:
using Calculus


f (generic function with 1 method)

In [5]:
f(x) = x^3 * exp(x) + sin(x)

f (generic function with 1 method)

Derivatives are shown below at x = 1.0

In [7]:
d = derivative(f, 1.0);
d2 = second_derivative(f,1.0);

println(d)
print(d2)

11.413429620197812
34.49618758929225

For functions with multiple variables, we compute the gradient and hessian. We consider

$$g(x) = (x_1)^2sin(3x_2)+e^{-2x_3}$$

In [8]:
g(x) = (x[1])^2 * sin(3x[2]) + exp(-2x[3])

g (generic function with 1 method)

The gradient at $x = [3.0, 1.0, 2.0]$

In [10]:
Calculus.gradient(g, [3.0, 1.0, 2.0])

3-element Vector{Float64}:
   0.8467200483621847
 -26.729797406378975
  -0.03663127778181647

In [11]:
hessian(g, [3.0,1.0,2.0])

3×3 Matrix{Float64}:
   0.282241  -17.8199  0.0
 -17.8199    -11.4307  0.0
   0.0         0.0     0.0732632

## 4.3 Numerical Integration

We will be using the QuadGK package which particularly uses the *Gauss-Kronrod Integration Method*. Suppose we integrate the following equation:
$$f(x) = -cos(3x) + x^2e^{-x}$$
over the interval $[0.0,1.0]$

In [13]:
f(x) = -cos(3x) + x^2 * exp(-x) 

f (generic function with 1 method)

In [15]:
using QuadGK
quadgk(f, 0.0,1.0)

(0.11356279145616596, 2.1219137558148304e-14)

The value for the area below the curve is $0.1136$ while the error is $2.1219e-14$